In [1]:
import sys
sys.path.append('/home/nick/python/asop_global/ASoP-Coherence')
import asop_coherence as asop
import iris
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from numba import jit
from tqdm import tqdm
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units

In [2]:
def load_cmip6(asop_dict):
    from iris.util import unify_time_units
    from iris.experimental.equalise_cubes import equalise_attributes
    from iris.time import PartialDateTime

    cubelist = iris.load(str(asop_dict['dir'])+'/pr_3hr*.nc3') # Use NetCDF3 data to save compute time
    unify_time_units(cubelist)
    equalise_attributes(cubelist)
    cube = cubelist.concatenate_cube()
    cube.coord('time').bounds = None
    constraint = iris.Constraint(time = lambda cell: PartialDateTime(year=asop_dict['start_year']) <= cell <= PartialDateTime(year=asop_dict['stop_year']),longitude = lambda cell: 60 <= cell <= 90, latitude = lambda cell: 10 <= cell <= 40)
    cube = cube.extract(constraint)
    return(cube)

In [3]:
def get_asop_dict(key):
    datapath=Path('/media/nick/lacie_tb31/data_from_gill/CMIP6')
    if key == 'AWI':
        asop_dict={
            'dir': datapath/'AWI-CM-1-1-MR',
            'name': 'AWI',
            'start_year': 1990,
            'stop_year': 2014,
            'dt': 10800,
            'legend_name': 'AWI',
            'region': [-90,90,0,360],
            'color': 'red',
            'symbol': '<'
        }
    elif key == 'BCC':
        asop_dict={
            'dir': datapath/'BCC-CSM2-MR',
            'name': 'BCC',
            'dt': 10800,
            'legend_name': 'BCC',
            'region': [-90,90,0,360],
            'color': 'blue',
            'symbol': '8'
        }
    else:
        raise Exception('No dictionary for '+key)
    return(asop_dict)

In [4]:
regions = [ ([-30,30,0,360],'land','trop_land'),
            ([-30,30,0,360],'ocean','trop_ocean'),
            ([-90,-30,0,360],'land','sh_land'),
            ([-90,-30,0,360],'ocean','sh_ocean'),
            ([30,90,0,360],'land','nh_land'),
            ([30,90,0,360],'ocean','nh_ocean'),
            ([-90,90,0,360],'land','glob_land'),
            ([-90,90,0,360],'ocean','glob_ocean')]
datasets=['AWI'] #,'BCC']
n_datasets=len(datasets)
n_regions = len(regions)
space_metrics_plot = np.empty((n_datasets,n_regions))
time_metrics_plot = np.empty((n_datasets,n_regions))
all_datasets = []
all_colors = []
all_symbols = []
all_regions = []
for box,mask_type,region_name in regions:
	all_regions.append(region_name)

In [5]:
def compute_clim_contribution(precip):
    import iris.coord_categorisation
    iris.coord_categorisation.add_month_number(precip,'time')
    month_clim = precip.aggregated_by('month_number',iris.analysis.SUM)
    ann_clim = month_clim.collapsed('time',iris.analysis.SUM)
    month_frac = month_clim/ann_clim
    month_frac.var_name='clim_precip_frac'
    month_frac.long_name='Fractional monthly precipitation contribution to annual precipitation'
    return(month_frac)

In [6]:
#for model in datasets:
model = 'AWI'
asop_dict = get_asop_dict(model)
precip = load_cmip6(asop_dict)
#precip.convert_units('mm day-1')

In [7]:
clim_contrib_filename='pr_clim_month_frac.'+str(asop_dict['start_year'])+'-'+str(asop_dict['stop_year'])+'.nc'
clim_contrib_file=asop_dict['dir']/clim_contrib_filename
if clim_contrib_file.exists():
    clim_contrib = iris.load_cube(str(clim_contrib_file))
else:
    clim_contrib = compute_clim_contribution(precip)
    iris.save(clim_contrib,str(clim_contrib_file))

In [8]:
def find_wet_season(precip,clim_contrib,threshold=1.0/24.0):
    month_list = []
    for m,month in enumerate(clim_contrib.coord('month_number').points):
        if clim_contrib.data[m] > threshold:
            month_list.append(month)
    month_constraint = iris.Constraint(month_number=month_list) #PartialDateTime(month=month_list))
    wet_season_cube = precip.extract(month_constraint)
    return(wet_season_cube)

In [14]:

def compute_temporal_summary(precip,clim_contrib,ndivs,twod=False,min_precip_threshold=1/86400.0,wet_season_threshold=1.0/24.0):
    # Compute temporal summary metric only
    iris.coord_categorisation.add_month_number(precip,'time')
    lon_coord = precip.coord('longitude')
    lat_coord = precip.coord('latitude')
    nlon = len(lon_coord.points)
    nlat = len(lat_coord.points)
    lower_thresh = iris.cube.Cube(data=np.empty((nlat,nlon)),dim_coords_and_dims=[(lat_coord,0),(lon_coord,1)])
    lower_thresh.var_name='lower_threshold'
    lower_thresh.long_name='Lower (off) threshold based on '+str(ndivs)+' divisions'
    upper_thresh = lower_thresh.copy()
    upper_thresh.var_name='upper_threshold'
    upper_thresh.long_name='Upper (on) threshold based on '+str(ndivs)+' divisions'
    if twod:
        time_inter = lower_thresh.copy()
        time_inter.var_name='temporal_onoff_metric'
        time_inter.long_name='Temporal intermittency on-off metric based on '+str(ndivs)+' divisions'
        onon_freq = lower_thresh.copy()
        onon_freq.var_name='freq_onon'
        onoff_freq = lower_thresh.copy()
        onoff_freq.var_name='freq_onoff'
        offon_freq = lower_thresh.copy()
        offon_freq.var_name='freq_offon'
        offoff_freq = lower_thresh.copy()
        offoff_freq.var_name='freq_offoff'

    # Use cube slices to avoid loading all data into memory
    for t,t_slice in enumerate(precip.slices(['time'])):
        lat = t // nlon
        lon = t % nlon
        month_list = []
        print(lat,lon)
        for m,month in enumerate(clim_contrib.coord('month_number').points):
            if clim_contrib.data[m,lat,lon] > wet_season_threshold:
                month_list.append(month)
        if len(month_list) > 1:
            month_constraint = iris.Constraint(month_number=month_list)
            wet_season = t_slice.extract(month_constraint)
            #wet_season = find_wet_season(t_slice,clim_contrib[:,lat,lon])
            this_precip = wet_season.data[np.where(wet_season.data > min_precip_threshold)] 
            nt = np.size(this_precip)
            if nt > ndivs:
                this_lower,this_upper,this_offoff,this_offon,this_onoff,this_onon = compute_onoff_metric(wet_season.data,this_precip.data)
                lower_thresh.data[lat,lon] = this_lower
                upper_thresh.data[lat,lon] = this_upper
                offoff_freq.data[lat,lon] = this_offoff
                offon_freq.data[lat,lon] = this_offon
                onoff_freq.data[lat,lon] = this_onoff
                onon_freq.data[lat,lon] = this_onon
            else:
                lower_thresh.data[lat,lon] = np.nan
                upper_thresh.data[lat,lon] = np.nan
                offoff_freq.data[lat,lon] = np.nan
                offon_freq.data[lat,lon] = np.nan
                onoff_freq.data[lat,lon] = np.nan
                onon_freq.data[lat,lon] = np.nan
        else:
            lower_thresh.data[lat,lon] = np.nan
            upper_thresh.data[lat,lon] = np.nan
            offoff_freq.data[lat,lon] = np.nan
            offon_freq.data[lat,lon] = np.nan
            onoff_freq.data[lat,lon] = np.nan
            onon_freq.data[lat,lon] = np.nan

    time_inter = 0.5*((onon_freq+offoff_freq)-(onoff_freq+offon_freq))
    out_cubelist = [time_inter,onon_freq,onoff_freq,offon_freq,offoff_freq,lower_thresh,upper_thresh]
    return(out_cubelist)

In [9]:
@jit(nopython=True,parallel=True)
def compute_onoff_metric(wet_season,this_precip):
    lower_thresh = np.percentile(this_precip,25)
    upper_thresh = np.percentile(this_precip,75)
    upper_mask = np.where(wet_season >= upper_thresh,1,0)
    lower_mask = np.where(wet_season <= lower_thresh,1,0)
    non = np.sum(upper_mask)
    noff = np.sum(lower_mask)
    onon = upper_mask + np.roll(upper_mask,1)
    onon_count = np.count_nonzero(np.where(onon == 2,1,0))
    onoff = upper_mask + np.roll(lower_mask,1)
    onoff_count = np.count_nonzero(np.where(onoff == 2,1,0))
    offon = lower_mask + np.roll(upper_mask,1)
    offon_count = np.count_nonzero(np.where(offon == 2,1,0))
    offoff = lower_mask + np.roll(lower_mask,1)
    offoff_count = np.count_nonzero(np.where(offoff == 2,1,0))
    onon_freq = onon_count/float(non)
    onoff_freq = onoff_count/float(non)
    offon_freq = offon_count/float(noff)
    offoff_freq = offoff_count/float(noff)

    return(lower_thresh,upper_thresh,offoff_freq,offon_freq,onoff_freq,onon_freq)

In [15]:
print(precip)
print(clim_contrib)
time_inter = compute_temporal_summary(precip,clim_contrib,4,twod=True)
iris.save(time_inter,'asop_coherence_global_cmip6_timeinter.nc')

precipitation_flux / (kg m-2 s-1)   (time: 73048; latitude: 33; longitude: 33)
     Dimension coordinates:
          time                           x                -              -
          latitude                       -                x              -
          longitude                      -                -              x
     Attributes:
          Conventions: CF-1.7 CMIP-6.2
          NCO: 4.7.2
          activity_id: CMIP
          branch_method: standard
          branch_time_in_child: 0.0
          branch_time_in_parent: 54421.0
          cmor_version: 3.4.0
          comment: includes both liquid and solid phases
          data_specs_version: 01.00.27
          experiment: all-forcing simulation of the recent past
          experiment_id: historical
          external_variables: areacella
          forcing_index: 1
          frequency: 3hr
          further_info_url: https://furtherinfo.es-doc.org/CMIP6.AWI.AWI-CM-1-1-MR.historical.none...
          grid: All grid attribu